In [ ]:
import pandas as pd
import os
import sys

def leer_archivo(ruta_archivo: str):
    # Verificar que el archivo existe
    if not os.path.exists(ruta_archivo):
        sys.exit(f"❌ El archivo no existe: {ruta_archivo}")

    # Detectar la extensión
    extension = os.path.splitext(ruta_archivo)[1].lower()

    try:
        if extension == ".csv":
            df = pd.read_csv(
                ruta_archivo,
                thousands=",",         # ✅ interpreta comas como miles (2022, 20000)
                na_values=["", " "],   # ✅ vacíos como NaN
                keep_default_na=True
            )
        elif extension in [".xls", ".xlsx"]:
            df = pd.read_excel(ruta_archivo)
        elif extension == ".json":
            df = pd.read_json(ruta_archivo)
        else:
            sys.exit(f"⚠️ Tipo de archivo no soportado: {extension}")

        # Mostrar solo las primeras 10 filas
        print(df.head(10))
        return df

    except Exception as e:
        sys.exit(f"❌ Error al leer el archivo: {e}")

In [ ]:
def combinar_dataframes(df1, df2, cols1=None, cols2=None, how="inner"):
    """
    Combina dos DataFrames en base a columnas comunes o especificadas.
    
    Parámetros:
    -----------
    df1 : DataFrame
        Primer DataFrame
    df2 : DataFrame
        Segundo DataFrame
    cols1 : str o list, opcional
        Columna(s) en df1 para la unión
    cols2 : str o list, opcional
        Columna(s) en df2 para la unión
    how : str, default "inner"
        Tipo de unión: 'left', 'right', 'outer', 'inner'

    Retorna:
    --------
    DataFrame combinado
    """
    if cols1 is None and cols2 is None:
        # Detectar columnas comunes automáticamente
        comunes = list(set(df1.columns) & set(df2.columns))
        if not comunes:
            raise ValueError("❌ No hay columnas en común para combinar.")
        return pd.merge(df1, df2, on=comunes, how=how)

    elif cols1 is not None and cols2 is not None:
        # Unión con columnas diferentes
        return pd.merge(df1, df2, left_on=cols1, right_on=cols2, how=how)

    else:
        raise ValueError("❌ Debes pasar ambas listas de columnas (cols1 y cols2) o ninguna.")


In [ ]:
import matplotlib.pyplot as plt

# Ruta del archivo
ruta = "./data/num_ev.csv"
df = leer_archivo(ruta)

In [ ]:
# ---------------------------
# Limpieza de columnas numéricas
# ---------------------------
# Asegurar que algunas columnas sean numéricas
cols_numericas = ["AÑO_REGISTRO", "CAPACIDAD_PASAJEROS", "POTENCIA"]
for col in cols_numericas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [ ]:
# ---------------------------
# 1. Vehículos registrados por año
# ---------------------------
conteo_anual = df["AÑO_REGISTRO"].value_counts().sort_index()
plt.figure(figsize=(8, 5))
conteo_anual.plot(kind="bar")
plt.title("Número de vehículos eléctricos registrados por año")
plt.xlabel("Año de registro")
plt.ylabel("Cantidad de vehículos")
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------
# 2. Distribución por marca (Top 10)
# ---------------------------
plt.figure(figsize=(8, 5))
df["MARCA"].value_counts().head(10).plot(kind="bar")
plt.title("Top 10 marcas de vehículos eléctricos")
plt.xlabel("Marca")
plt.ylabel("Cantidad de vehículos")
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------
# 3. Clasificación por clase (Automóvil, Moto, Bus…)
# ---------------------------
plt.figure(figsize=(6, 6))
df["CLASE"].value_counts().plot(kind="pie", autopct='%1.1f%%')
plt.title("Distribución por clase de vehículo")
plt.ylabel("")  # Quitar etiqueta lateral
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------
# 4. Potencia promedio por marca (Top 10)
# ---------------------------
if "POTENCIA" in df.columns:
    potencia_marca = df.groupby("MARCA")["POTENCIA"].mean().sort_values(ascending=False).head(10)
    plt.figure(figsize=(10, 5))
    potencia_marca.plot(kind="bar")
    plt.title("Potencia promedio por marca (Top 10)")
    plt.xlabel("Marca")
    plt.ylabel("Potencia (HP o kW)")
    plt.tight_layout()
    plt.show()

In [ ]:
# ---------------------------
# 5. Capacidad de pasajeros por tipo de servicio
# ---------------------------
if "CAPACIDAD_PASAJEROS" in df.columns and "SERVICIO" in df.columns:
    plt.figure(figsize=(7, 5))
    df.groupby("SERVICIO")["CAPACIDAD_PASAJEROS"].mean().plot(kind="bar")
    plt.title("Capacidad promedio de pasajeros por servicio")
    plt.xlabel("Servicio")
    plt.ylabel("Pasajeros promedio")
    plt.tight_layout()
    plt.show()